In [1]:
import copy
from typing import Set

import numpy as np

from src.formalisms.primitives import State, Action

In [2]:
from src.formalisms.policy import FinitePolicyForFixedCMDP
from src.solution_methods.linear_programming.cplex_dual_cmdp_solver import get_policy_solution_to_FiniteCMDP
from src.concrete_decision_processes.maze_cmdp import RoseMazeCMDP

cmdp = RoseMazeCMDP()
given_policy: FinitePolicyForFixedCMDP = get_policy_solution_to_FiniteCMDP(cmdp, True, False)[0]

Entering cplex: view dual_mdp_result_20230310_113816.log for info
Exiting cplex


In [3]:
given_policy.policy_matrix

array([[0.   , 0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   , 0.   ],
       [0.   , 0.686, 0.   , 0.314],
       [0.   , 0.   , 0.   , 1.   ],
       [0.   , 0.   , 0.   , 1.   ],
       [0.   , 0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   ]])

For the sake of testing the algorithm, we're going to make the policy stochastic on states that $\sigma$ never actually enters.

In [4]:
new_policy_matrix = given_policy.policy_matrix.copy()
null_row_mask = given_policy.occupancy_measure_matrix.sum(axis=1) == 0.0
new_policy_matrix[null_row_mask, :] = 0.0
new_policy_matrix[null_row_mask, 0] = 0.42
new_policy_matrix[null_row_mask, 1] = 0.58
new_policy_matrix

array([[0.   , 0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   , 0.   ],
       [0.   , 0.686, 0.   , 0.314],
       [0.   , 0.   , 0.   , 1.   ],
       [0.   , 0.   , 0.   , 1.   ],
       [0.42 , 0.58 , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   ]])

In [5]:
sigma = FinitePolicyForFixedCMDP.fromPolicyMatrix(cmdp, new_policy_matrix, True)

In [6]:
def A(policy: FinitePolicyForFixedCMDP, s: State) -> Set[Action]:
    return set(a for a in policy.A if policy(s).get_probability(a) > 0.0)

In [7]:
A(sigma, cmdp.state_list[2])
sigma.policy_matrix[2, :], sigma.occupancy_measure_matrix[2, :], np.isclose(sigma.policy_matrix[2, :], 0)

(array([0.   , 0.686, 0.   , 0.314]),
 array([0.   , 0.686, 0.   , 0.314]),
 array([ True, False,  True, False]))

In [8]:
m = sum(len(A(sigma, s)) - 1 for s in cmdp.S)
m

2

In [9]:
def get_phi_1():
    phi_1_policy_matrix = sigma.policy_matrix.copy()
    is_stochastic_state_mask = ((phi_1_policy_matrix > 0).sum(axis=1) != 1)
    states_inds_to_be_split = np.where(is_stochastic_state_mask)[0]
    print(phi_1_policy_matrix)
    for s_ind in states_inds_to_be_split:
        action_probs = phi_1_policy_matrix[s_ind, :]
        first_action_ind = np.where(action_probs > 0.0)[0][0]
        new_action_probs = np.zeros(cmdp.n_actions)
        new_action_probs[first_action_ind] = 1.0
        phi_1_policy_matrix[s_ind, :] = new_action_probs

    # Check row_stochastic
    assert np.allclose(phi_1_policy_matrix.sum(axis=1), 1.0), (phi_1_policy_matrix, phi_1_policy_matrix.sum(axis=1))
    # Check deterministic
    assert ((phi_1_policy_matrix > 0).sum(axis=1) == 1).all()
    return FinitePolicyForFixedCMDP.fromPolicyMatrix(cmdp=cmdp, policy_matrix=phi_1_policy_matrix)


phi_1 = get_phi_1()

[[0.    0.    0.    1.   ]
 [1.    0.    0.    0.   ]
 [0.    0.686 0.    0.314]
 [0.    0.    0.    1.   ]
 [0.    0.    0.    1.   ]
 [0.42  0.58  0.    0.   ]
 [1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [1.    0.    0.    0.   ]]


# Algorithm 1

## Inputs

In [10]:
sigma, phi_1,

(<src.formalisms.policy.FinitePolicyForFixedCMDP at 0x7fcda80dc250>,
 <src.formalisms.policy.FinitePolicyForFixedCMDP at 0x7fcdf834a490>)

## Initiation

### Line 1

$q^{\sigma}_{\mu}(x)$

In [11]:
sigma.state_occupancy_measure_vector

array([0.2826    , 0.40507614, 1.        , 0.500094  , 0.55566   ,
       0.        , 0.4500846 , 0.6174    , 6.18908526])

$Q^{\sigma}_{\mu}(x, a)$

In [12]:
sigma.occupancy_measure_matrix

array([[0.        , 0.        , 0.        , 0.2826    ],
       [0.40507614, 0.        , 0.        , 0.        ],
       [0.        , 0.686     , 0.        , 0.314     ],
       [0.        , 0.        , 0.        , 0.500094  ],
       [0.        , 0.        , 0.        , 0.55566   ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.4500846 , 0.        , 0.        , 0.        ],
       [0.        , 0.6174    , 0.        , 0.        ],
       [6.18908526, 0.        , 0.        , 0.        ]])

### Line 2

$q(x) \leftarrow q^{\sigma}(x)$

In [13]:
q = {
    cmdp.state_list[s_ind]: sigma.state_occupancy_measure_vector[s_ind]
    for s_ind in range(cmdp.n_states)
}

$A^*(x) \leftarrow A^{\sigma}(x)$
I'm not really sure how to interpret $A^*$

In [14]:
 A_star = {
    s: A(sigma, s)
    for s in cmdp.S
}
A_star

{XYState(x=0, y=1): {<IntAction(1)>},
 XYState(x=1, y=2): {<IntAction(2)>},
 XYState(x=0, y=0): {<IntAction(0)>, <IntAction(1)>},
 XYState(x=2, y=1): {<IntAction(1)>},
 XYState(x=2, y=0): {<IntAction(1)>},
 XYState(x=1, y=1): {<IntAction(0)>, <IntAction(2)>},
 XYState(x=2, y=2): {<IntAction(2)>},
 XYState(x=1, y=0): {<IntAction(0)>},
 XYState(x=0, y=2): {<IntAction(2)>}}

$\phi = \phi^1$

In [15]:
phi = copy.deepcopy(phi_1)

$j \leftarrow 1$

In [16]:
j = 1

U is the set of states that $\sigma$ won't actually visit, but on which $A^*$ has more than one action.
$U \leftarrow \{ x \in X \mid \   |A^*(x)| > 1, q(x)=0\}$

$V \leftarrow \{ x \in X \mid \   |A^*(x)| > 1, q(x)>0\}$

In [17]:
def U():
    return {
        s
        for s in cmdp.S
        if len(A_star[s]) > 1 and q[s] == 0
    }


def V():
    return {
        s
        for s in cmdp.S
        if len(A_star[s]) > 1 and q[s] > 0
    }


U(), V()

({XYState(x=1, y=1)}, {XYState(x=0, y=0)})

$Q(x, a) \leftarrow Q^{\sigma}_{\mu}(x,a)$ for $x \in V$ and $a
\in A^*(x)$

In [18]:
Q = {
    (s, a): sigma.occupancy_measure_matrix[cmdp.state_to_ind_map[s], cmdp.action_to_ind_map[a]]
    for s in V()
    for a in A_star[s]
}
Q

{(XYState(x=0, y=0), <IntAction(0)>): 0.6859999999999999,
 (XYState(x=0, y=0), <IntAction(1)>): 0.314}

In [19]:
def get_updated_deterministic_policy(
        policy: FinitePolicyForFixedCMDP,
        s: State,
        a: Action
) -> FinitePolicyForFixedCMDP:
    s_ind = policy.cmdp.state_to_ind_map[s]
    a_ind = policy.cmdp.action_to_ind_map[a]
    modified_policy_matrix = policy.policy_matrix.copy()
    modified_policy_matrix[s_ind, :] = 0.0
    modified_policy_matrix[s_ind, a_ind] = 1.0
    return FinitePolicyForFixedCMDP.fromPolicyMatrix(policy.cmdp, modified_policy_matrix)


In [20]:
from src.formalisms.distributions import KroneckerDistribution
import random

print(U())
alphas = [None] * 10
phis = [None] * 10
phis[1] = phi_1
while len(U()) > 0:
    z = random.sample(U(), 1)[0]
    print(z)
    phi_j_of_z = phis[j](z).sample()
    potential_new_actions = A_star[z] - {phi_j_of_z}
    a = random.sample(potential_new_actions, 1)[0]
    alphas[j] = 0
    phis[j + 1] = FinitePolicyForFixedCMDP.fromPolicyDict(cmdp, {
        s: phis[j](s) if s != z else KroneckerDistribution(a)
        for s in cmdp.S
    })
    A_star[z] = A_star[z] - {phi_j_of_z}
    phi = get_updated_deterministic_policy(phi, z, a)
    j = j + 1

{XYState(x=1, y=1)}
XYState(x:1, y:1, )


In [21]:
list(map((lambda item: None if item is None else item.policy_matrix), phis))

[None,
 array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]),
 array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]),
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [22]:
_s = random.sample(V(), 1)[0]
sigma.policy_matrix[cmdp.state_to_ind_map[_s]]

array([0.   , 0.686, 0.   , 0.314])

### Recursive Step

In [23]:
while len(V()) > 0:
    candidate_ajs = {
        s: Q[(s, phi.get_deterministic_action(s))] / phi.get_state_occupancy_measure(s)
        for s in list(V())
    }

    alphas[j] = min(candidate_ajs.values())

    G = [
        s
        for s in candidate_ajs.keys()
        if candidate_ajs[s] == alphas[j]

    ]
    k = len(G)
    print(phis[j])
    print(phis[j+1])
    for i in range(1, k+1):
        prev_phi = phis[j + i - 1]
        s_i = G[i - 1]
        action_choice = random.sample(A(sigma, s_i) - {prev_phi.get_deterministic_action(s_i)}, 1)[0]
        next_phi = get_updated_deterministic_policy(prev_phi, s_i, action_choice)
        phis[j+i] = next_phi

    for i in range(1, k-1+1):
        alphas[j+i] = 0

    for s in G:
        A_star[s].remove(phi.get_deterministic_action(s))
        phi = get_updated_deterministic_policy(phi, s, phis[j+k].get_deterministic_action(s))

    j = j + k
    for x in V():
        Q[(x, phi.get_deterministic_action(x))] -= alphas[j] * phi.get_occupancy_measure(x, phi.get_deterministic_action(x))

candidate_ajs, G

None


({XYState(x=0, y=0): 0.6859999999999999}, [XYState(x=0, y=0)])

In [24]:
m = j - 1
alphas[m+1] = 1 - sum(alphas[1:m+1])

In [25]:
m

2

In [26]:
phis[2:m+2]

In [27]:
alphas[1:m+2]

[0, 0.6859999999999999, 0.31400000000000006]

In [29]:
combined_occ_measures = sum([
    alphas[j] * phis[j].occupancy_measure_matrix
    for j in range(1, m+2)
])
combined_occ_measures

array([[0.        , 0.        , 0.        , 0.2826    ],
       [0.40507614, 0.        , 0.        , 0.        ],
       [0.        , 0.686     , 0.        , 0.314     ],
       [0.        , 0.        , 0.        , 0.500094  ],
       [0.        , 0.        , 0.        , 0.55566   ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.4500846 , 0.        , 0.        , 0.        ],
       [0.        , 0.6174    , 0.        , 0.        ],
       [6.18908526, 0.        , 0.        , 0.        ]])

In [32]:
assert np.allclose(combined_occ_measures, sigma.occupancy_measure_matrix)